# AWS Lab 1 Summarize Dialogue

In [1]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hf_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(hf_dataset_name)

In [3]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))
print(dataset['test'][0].keys())

for i, index in enumerate(example_indices):

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print('Input Dialog: ')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('Human Summary: ', end="")
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

dict_keys(['id', 'dialogue', 'summary', 'topic'])
---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Dialog: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Human Summary: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------

In [4]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
sentence = "What time is it, Tom?"

sentece_encoded = tokenizer.encode(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(sentece_encoded[0], skip_special_tokens=True)

print("Encoded sentence: ", sentece_encoded)

print("Decoded token: ", sentence_decoded)

Encoded sentence:  tensor([[ 363,   97,   19,   34,    6, 3059,   58,    1]])
Decoded token:  What time is it, Tom?


In [6]:
# Define a printing helper function
def print_output(dialogue, summary, model_output):
    print("Dialoge: ")
    print(dialogue)
    print(dash_line)
    print("Summary: ", end="")
    print(summary)
    print("Model Output: ", end="")
    print(model_output)
    print(dash_line)
    print()


In [7]:
# See Model output without any prompt

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    tokenized_dialogue = tokenizer(dialogue, return_tensors='pt')

    model_output = tokenizer.decode(model.generate(tokenized_dialogue['input_ids'], max_new_tokens=50)[0], skip_special_tokens=True)
    print_output(dialogue, summary, model_output)
    


Dialoge: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Summary: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
Model Output: Person1: It's ten to nine.
---------------------------------------------------------------------------------------------------

Dialoge: 
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would

## Zero Shot Inference with an Instruction Prompt
### See model output with prompt to summarize the given input

In [8]:
# Zero Shot Inference with an Instruction Prompth
# See model output with prompt to summarize the given input

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    # input_prompt = prompt + dialogue
    input_prompt = f"""
    Summarize the following conversation:
    {dialogue}
    Summary: 
    """
    tokenized_dialogue = tokenizer(input_prompt , return_tensors='pt')

    model_output = tokenizer.decode(model.generate(tokenized_dialogue['input_ids'], max_new_tokens=50)[0], skip_special_tokens=True)
    print_output(input_prompt, summary, model_output)

Dialoge: 

    Summarize the following conversation:
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    Summary: 
    
---------------------------------------------------------------------------------------------------
Summary: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
Model Output: The train is about to leave.
---------------------------------------------------------------------------------------------------

Dialoge: 

    Summarize the following conversation:
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program

## 4. Summarize Dialogue with One Shot and Few Shot Inference

Build a function to generate prompt that end with a prompt that you want the model to complete

In [9]:
def make_prompt(list_idx, dataset, idx_to_sum):
    output_prompt = ""
    for i, index in enumerate(list_idx):
        dialogue = dataset[index]['dialogue']
        summary = dataset[index]['summary']
        prompt = f"""Dialogue: 
                        {dialogue}
                     What was going on? 
                        {summary}
                    """  
        output_prompt += prompt

    output_prompt += f"Dialoge: {dataset[idx_to_sum]['dialogue']} \n What was going on? " 
    return output_prompt

print(make_prompt([40], dataset['test'], 200))


Dialogue: 
                        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
                     What was going on? 
                        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
                    Dialoge: #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that

### 4.1 One Shot Learning

In [10]:
input_prompt = make_prompt([40], dataset['test'], 200)
tokenized_input = tokenizer(input_prompt, return_tensors='pt')
model_output = tokenizer.decode(model.generate(tokenized_input['input_ids'], max_new_tokens=50)[0], skip_special_tokens=True)

print_output(input_prompt, dataset['test'][200]['summary'], model_output)


Dialoge: 
Dialogue: 
                        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
                     What was going on? 
                        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
                    Dialoge: #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can 

### 4.2 Few Shot Inference

In [11]:
input_prompt = make_prompt([40, 80, 120], dataset['test'], 200)
tokenized_input = tokenizer(input_prompt, return_tensors='pt')
model_output = tokenizer.decode(model.generate(tokenized_input['input_ids'], max_new_tokens=50)[0], skip_special_tokens=True)

print_output(input_prompt, dataset['test'][200]['summary'], model_output)


Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


Dialoge: 
Dialogue: 
                        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
                     What was going on? 
                        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
                    Dialogue: 
                        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for

## 5. Generative Configuration Parameters for Inference

Change the configuration parameters of the `generate()` method to see different output from the LLM. A full list of available parameters can be found in the [Hugging Face Generation documentation](https://huggingface.co/docs/transformers/en/main_classes/text_generation).

In [12]:
temperature=[0.1, 1.0, 2.0, 3.0]
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

input_prompt = make_prompt([40, 80, 120], dataset['test'], 200)
tokenized_input = tokenizer(input_prompt, return_tensors='pt')
print("Baseline Human Summary: ", dataset['test'][200]['summary'])
print()

for temp in temperature:
    print("Temperature: ", temp)
    generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=temp)
    for i in range(5):
        model_output = tokenizer.decode(model.generate(tokenized_input['input_ids'], generation_config=generation_config)[0], skip_special_tokens=True)
        print(i+1, model_output)
    print(dash_line)


Baseline Human Summary:  #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

Temperature:  0.1
1 #Person1 wants to upgrade his computer. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.
2 #Person1 wants to upgrade his computer.
3 #Person1 wants to upgrade his computer and hardware.
4 #Person1 wants to upgrade his computer.
5 #Person1 wants to upgrade his computer.
---------------------------------------------------------------------------------------------------
Temperature:  1.0
1 At this moment, Person1 is choosing a new computer system to upgrade.
2 People can buy a new computer through an internet browser or mobile phone, as most software projects are coming out on Cds today.
3 #Replacement is a program that will make user's life easier.
4 #Person2 recommends some options about upgrading a computer.
5 At the moment, Person1 has to decide which software to choose from.
-----------------------------